In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
testdict = {'name': ['A', 'B', 'C'], 'int': ['1', '2', '3']}

df = pd.DataFrame.from_dict(testdict)

In [3]:
print(df)

  name int
0    A   1
1    B   2
2    C   3
